In [1]:
import os
os.chdir("../")

In [2]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class PrepareBaseModelConfig:
    root_dir: Path
    model_path: Path
    params_image_size: list
    params_learning_rate: float
    params_classes: int


In [3]:
from cnnClassifier.constant import *
from cnnClassifier.utils.common import read_yaml, create_directories

In [4]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_PATH,
        params_filepath = PARAMS_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    def get_prepare_base_model_config(self) -> PrepareBaseModelConfig:
        config = self.config.prepare_base_model

        create_directories([config.root_dir])

        prepare_base_model_config = PrepareBaseModelConfig(
            root_dir=Path(config.root_dir),
            model_path=Path(config.model_path),
            params_image_size=self.params.IMAGE_SIZE,
            params_learning_rate=self.params.LEARNING_RATE,
            params_classes=self.params.CLASSES
        )

        return prepare_base_model_config

In [5]:
import os
import urllib.request as request
import zipfile
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Conv2D, BatchNormalization, Dropout, Flatten, MaxPooling2D, Input, Activation, Dense

2023-09-22 23:52:11.975098: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-09-22 23:52:11.999437: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-22 23:52:12.485267: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [6]:
class PrepareBaseModel:
    
    def __init__(self, config: PrepareBaseModelConfig):
        self.config = config
        self.input_shape = self.config.params_image_size
        self.num_classes = self.config.params_classes
        self.model = self.prepare_model()

    #@staticmethod    
    def prepare_model(self):
        self.model = Sequential()
        self.model.add(keras.layers.Conv2D(32, (2, 2), activation='relu', input_shape=self.input_shape, kernel_initializer = 'he_normal'))
        self.model.add(keras.layers.MaxPooling2D((3, 3), strides=(2, 2), padding='same'))
        self.model.add(keras.layers.BatchNormalization())

        # 2nd conv layer
        self.model.add(keras.layers.Conv2D(32, (2, 2), activation='relu', kernel_initializer = 'he_normal'))
        self.model.add(keras.layers.MaxPooling2D((3, 3), strides=(2, 2), padding='same'))
        self.model.add(keras.layers.BatchNormalization())

        # 3rd conv layer
        self.model.add(keras.layers.Conv2D(32, (2, 2), activation='relu', kernel_initializer = 'he_normal'))
        self.model.add(keras.layers.MaxPooling2D((2, 2), strides=(2, 2), padding='same'))
        self.model.add(keras.layers.BatchNormalization())

        # flatten output and feed it into dense layer
        self.model.add(keras.layers.Flatten())
        self.model.add(keras.layers.Dense(128, activation='relu', kernel_initializer = 'he_normal'))
        self.model.add(keras.layers.Dropout(0.5))
        self.model.add(keras.layers.Dense(64, activation='relu', kernel_initializer = 'he_normal'))
        self.model.add(keras.layers.Dropout(0.5))
        # output layer
        self.model.add(keras.layers.Dense(self.num_classes, activation='softmax'))
        
        self.model.compile(
            optimizer=tf.keras.optimizers.SGD(learning_rate=self.config.params_learning_rate),
            loss=tf.keras.losses.CategoricalCrossentropy(),
            metrics=["accuracy"]
        )

        self.model.summary()
        #return self.model       

        self.save_model(path=self.config.model_path, model=self.model)

    
    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        model.save(path)


In [8]:
try:
    config = ConfigurationManager()
    prepare_base_model_config = config.get_prepare_base_model_config()
    prepare_base_model = PrepareBaseModel(config=prepare_base_model_config)
except Exception as e:
    raise e

[2023-09-22 23:53:44,223]: INFO: common: yaml file: config/config.yaml loaded successfully
[2023-09-22 23:53:44,224]: INFO: common: yaml file: params.yaml loaded successfully
[2023-09-22 23:53:44,224]: INFO: common: created directory at: artifacts
[2023-09-22 23:53:44,224]: INFO: common: created directory at: artifacts/prepare_base_model
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 215, 12, 32)       160       
                                                                 
 max_pooling2d (MaxPooling2  (None, 108, 6, 32)        0         
 D)                                                              
                                                                 
 batch_normalization (Batch  (None, 108, 6, 32)        128       
 Normalization)                                                  
                                              

2023-09-22 23:53:44.238736: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-09-22 23:53:44.254715: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-09-22 23:53:44.254865: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf